In [285]:
##pip install spacy
##python -m spacy download en_core_web_sm

### Search: rules for glossing in ASL 
ChatGPT: what are the rules for glossing in ASL

In [2]:
import spacy

import re

from nltk import pos_tag
from nltk.corpus import wordnet, stopwords
stop_words = stopwords.words("english")
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [3]:
def cleanre(x):
    y=x
    y=re.sub(r'@\w+', '', y)
    y=re.sub(r'https(.*)', '', y)
    y=re.sub(r'#(.*)', '', y)
    return y

In [4]:
print(f'stop_words: {stop_words}')

## custom stop_words as we want to keep pronouns
stop_words = ['a', 'to', 'the', 'as', 'about']

def process(document):
    document = cleanre(document)
    tokens = word_tokenize(document)
    ##tokens = [t.lower() for t in tokens if t.isalpha()]
    tokens = [t for t in tokens if t not in stop_words]
    ###tokens = [stemmer.stem(t) for t in tokens]
    ##tokens = [lemmatizer.lemmatize(t) for t in tokens]
    ###tokens = [t for t in tokens if t not in cleaners]

    return " ".join(tokens)

stop_words: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so

In [5]:
convert_poss_pronouns = {'HER': 'SHE', 'HIS': 'HE', 'THEIR': 'THEY', 'THEIRS': 'THEY', 'YOURS': 'YOU', 'OURS': 'WE'}

opened_questions = ['WHO', 'WHAT', 'WHY', 'WHERE', 'WHOSE']

time_adverbs = ['YESTERDAY', 'TODAY', 'TOMORROW', 'SOON']
days = [day.upper() for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']]
print(days)
time_adverbs = time_adverbs + days
print(time_adverbs)

['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY', 'SUNDAY']
['YESTERDAY', 'TODAY', 'TOMORROW', 'SOON', 'MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY', 'SUNDAY']


In [6]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def translate_to_asl_gloss(text):
    # Preprocess the english text
    text = process(text) ## voir dans spacy nlp !!
    ###print(f'text: {text}')

    # Process the text with spaCy
    doc = nlp(text)

    # Initialize an empty list for the ASL gloss tokens
    asl_gloss_tokens = []

    # Define simple rules for transformation based on POS-tags
    for token in doc:
        #if token.pos_ == 'PRON' and token.dep_ == 'nsubj':
        #    # Move the subject pronoun to the end
        #    continue
        if token.pos_ == 'AUX':
            # Skip auxiliary verbs (like 'is', 'am', 'are')
            continue
        elif token.dep_ == 'punct':
            # Skip punctuation
            continue
        else:
            ## Use token.lemma_ not token.text
            ##formatted_token = token.text.upper()
            formatted_token = token.lemma_.upper()
            # Hours formatting to gloss
            if 'P.M' in formatted_token or 'A.M' in formatted_token:
                formatted_token = 'DESC-' + formatted_token
            # Pronoun not subject: X-
            elif token.pos_ == 'PRON':
                if formatted_token in convert_poss_pronouns.keys():
                    formatted_token = convert_poss_pronouns[formatted_token]
                formatted_token = 'X-' + formatted_token
            elif token.pos_ == 'RB':
                formatted_token = 'DESC-' + formatted_token
            ## opened questions: append 'Wh'
            if asl_gloss_tokens == []:
                if formatted_token in opened_questions:
                    formatted_token = 'wh ' + formatted_token
                elif '?' in text:
                    formatted_token = 'q ' + formatted_token

            asl_gloss_tokens.append(formatted_token)

    # Move the time adverb at start of list
    if asl_gloss_tokens[-1] in time_adverbs:
        asl_gloss_tokens.insert(0,asl_gloss_tokens.pop(asl_gloss_tokens.index(asl_gloss_tokens[-1])))

    # Join the tokens to form the ASL gloss
    asl_gloss = ' '.join(asl_gloss_tokens)
    return asl_gloss

## Dataset

In [7]:
# Example sentences
sentences = [
    "the high costs of patents in europe , you say , might possibly explain this",
    "i should like to congratulate you once again , commissioner , on a fascinating publication",
    "i will also create the post of commissioner for internal affairs and migration , including security",
    "the sitting was suspended at 3.25 p.m. and resumed at 6.00 p.m.",
    "Where is the bathroom?",
    "pl mr president , I would like to thank the rapporteur for her work",
    "Do you like coffee ?",
    "I will go to the store tomorrow."
]

expected_glosses = [
    "DESC-HIGH COST PATENT IN EUROPE , X-YOU SAY , MIGHT DESC-POSSIBLY EXPLAIN THIS",
    "X-I SHOULD LIKE TO CONGRATULATE X-YOU DESC-ONCE DESC-AGAIN , COMMISSIONER , ON DESC-FASCINATING PUBLICATION",
    "X-I WILL DESC-ALSO CREATE POST COMMISSIONER FOR DESC-INTERNAL AFFAIR AND MIGRATION , INCLUDE SECURITY",
    "SIT BE SUSPEND AT 3.25 DESC-P.M. AND RESUME AT 6.00 DESC-P.M.",
    "wh WHERE BATHROOM",
    "PL MR PRESIDENT , X-I WOULD LIKE TO THANK RAPPORTEUR FOR X-SHE WORK",
    "q YOU LIKE COFFEE",
    "TOMORROW STORE I GO"
]

In [8]:
## text-to-ASL gloss translation
# Translate each sentence to ASL gloss
for idx, sentence in enumerate(sentences):
    print(f"Original      : {sentence}")
    print(f"expected gloss: {expected_glosses[idx]}")
    print(f"ASL Gloss     : {translate_to_asl_gloss(sentence)}\n")


Original      : the high costs of patents in europe , you say , might possibly explain this
expected gloss: DESC-HIGH COST PATENT IN EUROPE , X-YOU SAY , MIGHT DESC-POSSIBLY EXPLAIN THIS
ASL Gloss     : HIGH COST OF PATENT IN EUROPE X-YOU SAY POSSIBLY EXPLAIN X-THIS

Original      : i should like to congratulate you once again , commissioner , on a fascinating publication
expected gloss: X-I SHOULD LIKE TO CONGRATULATE X-YOU DESC-ONCE DESC-AGAIN , COMMISSIONER , ON DESC-FASCINATING PUBLICATION
ASL Gloss     : X-I LIKE CONGRATULATE X-YOU ONCE AGAIN COMMISSIONER ON FASCINATING PUBLICATION

Original      : i will also create the post of commissioner for internal affairs and migration , including security
expected gloss: X-I WILL DESC-ALSO CREATE POST COMMISSIONER FOR DESC-INTERNAL AFFAIR AND MIGRATION , INCLUDE SECURITY
ASL Gloss     : X-I ALSO CREATE POST OF COMMISSIONER FOR INTERNAL AFFAIR AND MIGRATION INCLUDE SECURITY

Original      : the sitting was suspended at 3.25 p.m. and resumed

## nom propre
## règles standard